In [1]:
import torch
import random
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from MulticoreTSNE import MulticoreTSNE as TSNE

random.seed(42)

In [ ]:
# Construct the list of names
words = list()
with open("names.txt", "r") as infile:
    for line in infile:
        words.append(line.strip())
print(len(words))
print(max(len(w) for w in words))

In [3]:
# Mapping of characters to/from integers
special = "."
chars = sorted(list(set("".join(words))))
STOI = {s: i + 1 for i, s in enumerate(chars)}
STOI[special] = 0
ITOS = {i: s for s, i in STOI.items()}
vocab_size = len(STOI)

In [ ]:
def build_dataset(words, *, special: str = ".", block_size: int = 3) -> tuple[torch.tensor]:
    (X, Y) = (list(), list())

    for word in words:
        context = [0] * block_size
        for char in word + special:
            ix = STOI[char]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(f"Constructed dataset: X: {X.shape}, Y: {Y.shape}")
    return X, Y


# Train, Development, Test split
block_size = 3
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size=block_size)
Xdev, Ydev = build_dataset(words[n1:n2], block_size=block_size)
Xtest, Ytest = build_dataset(words[n2:], block_size=block_size)

In [5]:
# Utility function for comparing manually generated gradients to those calculated by pytorch


def cmp(s, dt, t):
    exact = torch.all(dt == t.grad).item()
    approx = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}")

In [ ]:
n_hidden = 64
n_embedding = 10

gen = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embedding), generator=gen)

# Layer 1
kaiming_init = (5 / 3) / ((n_embedding * block_size) ** 0.5)
W1 = torch.randn((block_size * n_embedding, n_hidden), generator=gen)
b1 = torch.randn(n_hidden, generator=gen)

# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=gen) * 0.1
b2 = torch.randn(vocab_size, generator=gen) * 0.1

# BatchNorm parameters
bn_gain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bn_bias = torch.randn((1, n_hidden)) * 0.1
bn_momentum = 0.001

parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]
print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

In [7]:
batch_size = 32
n = batch_size  # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=gen)
Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

In [ ]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb]  # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1  # hidden layer pre-activation

# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)  # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bn_raw = bndiff * bnvar_inv
hpreact = bn_gain * bn_raw + bn_bias

# Non-linearity
h = torch.tanh(hpreact)  # hidden layer

# Linear layer 2
logits = h @ W2 + b2  # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
    p.grad = None

for t in [
    logprobs,
    probs,
    counts,
    counts_sum,
    counts_sum_inv,
    norm_logits,
    logit_maxes,
    logits,
    h,
    hpreact,
    bn_raw,
    bnvar_inv,
    bnvar,
    bndiff2,
    bndiff,
    hprebn,
    bnmeani,
    embcat,
    emb,
]:
    t.retain_grad()
loss.backward()
loss

In [ ]:
"".join([ITOS[item.item()] for item in Ytr[:35]])

In [ ]:
for i, element in enumerate(Xb):
    batch = [ITOS[elem.item()] for elem in element]
    nxt = [ITOS[Yb[i].item()]]
    print("".join(batch + nxt))

In [9]:
# Construct manual backpropagation through the entire network

# dlogprobs - we want to compute the gradient of the loss with respect to all the elements of the dlogprobs tensor

# cmp('dlogprobs', dlogprobs, logprobs)